In [2]:
import numpy as np
from itertools import product, permutations
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from scipy.sparse import csr_matrix
import sys

# Matched Pairs Problem

Stable marriage problem

Minimum Weight Euclidean Matching (MWEM) 
Pefect matching between vectors in one group with another

## Weighted bipartite

In [2]:
c = np.array([[0,1,2,3],[10,11,12,13],[20,21,22,23],[30,31,32,33],[40,41,42,43],[50,51,52,53],[60,61,62,63]])
c

array([[ 0,  1,  2,  3],
       [10, 11, 12, 13],
       [20, 21, 22, 23],
       [30, 31, 32, 33],
       [40, 41, 42, 43],
       [50, 51, 52, 53],
       [60, 61, 62, 63]])

In [3]:
random_m = np.random.rand(500,10)
random_f = np.random.rand(500,10)

In [ ]:
matrix_m = np.array(range(6)).reshape(3,2)
matrix_f = np.array(range(7,13)).reshape(3,2)

print( 'MALE\n', matrix_m )
print( 'FEMALE\n', matrix_f )

Alternative to itertool's permutation function to create $\frac{n!}{k!(n-k)!}$ matches where k = 2. These pairings eliminate duplicate pairings (i.e. considers 1,2 the same as 2,1). This method was inefficient. DON'T USE.

In [ ]:
def cartesian_product( x, y ):
    # Faster on smaller datasets (100, 70) but slower on larger datasets (500, 700)
    return np.hstack([ np.repeat(x, y.shape[0], axis=0),
               np.tile(y, (x.shape[0],1))] ).reshape(-1, 2, x.shape[1])

Developed a custom method as the libraries in existance are limited to provide only one matching solution even if there are multiple solutions of equal likelihood. However, this method is not scalable (algorithmically and memory inefficient) and crashes my computer on medium-sized datasets (e.g. two matrices of shape (500, 10))

In [ ]:
def match_pairs_custom(matrix_a, matrix_b):
    """
    return: list of row indices
    type: tuple(np.array, range)
    """
    #pairs = np.array(list(product(random_m, random_f))) # as opposed to custom cartesian_product
    dist = cdist(matrix_a, matrix_b, 'euclidean')
    if dist.shape[0] != dist.shape[1]:
        return None
    n = dist.shape[0]
    combinations = np.array(list(permutations(range(n))))
    summation = np.choose(combinations, dist).sum(axis = 1) # Increment through columns and grab rows which corerspond to array
    return combinations[np.where(summation == summation.min())] # respective column indices are all range(n)

[Scipy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.min_weight_full_bipartite_matching.html) recommends linear sum assignment over minimized bipartite matchign as it is faster for dense graphs (which we have)

In [4]:
def match_pairs( matrix_a, matrix_b ):
    """
    Determine the most optimal unique pairings between observations in matrix a with matrix b.
    
    @param: matrix_a:
    @type: np.array
    @param: matrix_b
    @type: np.array
    @return: tuple of row indicies indicating pairings (matrix_a index order, matrix_b index order) 
    @type: tuple(np.array, np.array)
    """
    dist = cdist(matrix_a, matrix_b, 'euclidean')
    return linear_sum_assignment(dist)

[Hopcroft-Karp Algorithm](https://en.m.wikipedia.org/wiki/Hopcroft–Karp_algorithm)

$min(\sum_{i,j}d_{ij}x_{ij})\Longrightarrow O(E{\sqrt V})\approx O(V^\frac{3}{2})$ 

where $d_{ij}=distance~btw~V_i~and~V_j,$

$x_{ij}=\left\{\begin{matrix}1, & edge~belongs~to~matching \\ 0, & otherwise \end{matrix}\right.$

**Constraints**

1) Person $j$ paired with only one person from the other group: $\sum_jx_{ij}=1~for~1\le i\le n$

2) Person $i$ paired with only one person from the other group: $\sum_ix_{ij}=1~for~1\le j\le n$

3) Pairing between person $i$ and $j$ also means a pairing between $j$ and $i$: $x_{ij}\ge0 for 1\le i,j\le n$

In [ ]:
def match_pairs(matrix_a, matrix_b):
    n = matrix_a.shape[0] + matrix_b.shape[0]
    
    alpha = LpVariable("alpha", 1, n)
    beta = LpVariable("beta", 1, n)

[A Survey on Algorithms for Euclidean Matching](https://courses.cs.duke.edu/fall08/cps234/projects/sayan_proj.pdf)

$O(n^\frac{3}{2}log(n))$

$max(\sum_i\alpha_i+\sum_j\beta_j)$ subject to $\alpha_i+\beta_j\le d_{ij}$ for $1\le i,j\le n$

https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0
    
https://scaron.info/blog/linear-programming-in-python-with-pulp.html
        
https://towardsdatascience.com/how-to-match-two-people-with-python-7583b51ff3f9
            
https://medium.com/opex-analytics/optimization-modeling-in-python-pulp-gurobi-and-cplex-83a62129807a

In [222]:
from pulp import LpVariable, LpProblem, LpMaximize, value, lpSum

def match_pairs(matrix_a, matrix_b):
    dist = cdist(matrix_a, matrix_b, 'euclidean') # (matrix_a.shape[0], matrix_b.shape[0])
    prob = LpProblem("Matching", LpMaximize)
    
    variable_names = [str(i)+str(j) for i in range(matrix_a.shape[0]) for j in range(matrix_b.shape[0])]
    variable_names.sort()
    
    DV_variables = LpVariable.matrix( "X", variable_names, 1, matrix_a.shape[0], matrix_b.shape[0] )
    match = np.array(DV_variables).reshape(2,4)
    
    
    alpha = LpVariable("alpha", 1, n)
    beta = LpVariable("beta", 1, n)
    """for i in range(n):
        for j in range(n):
            prob += lpSum(alpha[i]) + lpSum(beta[j])
            prob += alpha[i] + beta[j] <= np.linalg.norm(alpha[i] - beta[j])
    """
    prob += lpSum(alpha) + lpSum(beta)
    prob += alpha + beta <= np.linalg.norm(alpha - beta)
    status = prob.solve()
    return value(alpha), value(beta)

Neither of these package functions were loading correctly. Therefore did math by hand.

## Non bipartite